In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# Joins results, races and drivers in a single dataset

In [14]:
results_df = pd.read_csv("data/formula-1/results.csv")
races_df = pd.read_csv("data/formula-1/races.csv")
drivers_df = pd.read_csv("data/formula-1/drivers.csv")
rr_df = pd.merge(results_df, races_df, on="raceId")
rrd = pd.merge(rr_df, drivers_df, on="driverId")

In [15]:
# Pilotos com mais pontos ao longo do tempo
# precisarei contabilizar as vitórias e aplicar a pontuação atual para ficar justo

In [17]:
rrd.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'name',
       'date', 'time_y', 'url_x', 'driverRef', 'number_y', 'code', 'forename',
       'surname', 'dob', 'nationality', 'url_y'],
      dtype='object')

In [27]:
rrd["position"].unique()

array(['1', '5', '13', '3', '2', '\\N', '10', '7', '12', '9', '4', '6',
       '16', '18', '14', '8', '19', '15', '17', '11', '23', '20', '21',
       '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33'],
      dtype=object)

In [30]:
rrd["grid"] = rrd["grid"].replace("\\N", "0")
rrd["position"] = rrd["position"].replace("\\N", "0")


In [36]:
rrd["delta_pos"] = rrd["grid"].astype("int64") - rrd["position"].astype("int64")
rrd[["name", "year", "forename", "surname", "grid", "position", "delta_pos"]]

,name,year,forename,surname,grid,position,delta_pos
0,Australian Grand Prix,2008,Lewis,Hamilton,1,1,0
1,Malaysian Grand Prix,2008,Lewis,Hamilton,9,5,4
2,Bahrain Grand Prix,2008,Lewis,Hamilton,3,13,-10
3,Spanish Grand Prix,2008,Lewis,Hamilton,5,3,2
4,Turkish Grand Prix,2008,Lewis,Hamilton,3,2,1
...,...,...,...,...,...,...,...
25135,Monaco Grand Prix,2021,Nikita,Mazepin,19,17,2
25136,Azerbaijan Grand Prix,2021,Nikita,Mazepin,18,14,4
25137,French Grand Prix,2021,Nikita,Mazepin,18,20,-2
25138,Styrian Grand Prix,2021,Nikita,Mazepin,20,18,2


In [37]:
rrd_2020 = rrd[rrd["year"] == 2020]
rrd_2020[["name", "year", "forename", "surname", "grid", "position", "delta_pos"]]

,name,year,forename,surname,grid,position,delta_pos
247,Austrian Grand Prix,2020,Lewis,Hamilton,5,4,1
251,Styrian Grand Prix,2020,Lewis,Hamilton,1,1,0
252,Hungarian Grand Prix,2020,Lewis,Hamilton,1,1,0
253,British Grand Prix,2020,Lewis,Hamilton,1,1,0
254,70th Anniversary Grand Prix,2020,Lewis,Hamilton,2,2,0
...,...,...,...,...,...,...,...
25099,Sakhir Grand Prix,2020,Nicholas,Latifi,16,0,16
25100,Abu Dhabi Grand Prix,2020,Nicholas,Latifi,18,17,1
25110,Sakhir Grand Prix,2020,Jack,Aitken,17,16,1
25111,Sakhir Grand Prix,2020,Pietro,Fittipaldi,20,17,3
